In [1]:
import datetime

import intervaltree

import db

In [2]:
ephem = db.getpasses('ephem-passes.db', gs='KB9JHU')
orbital = db.getpasses('orbital-passes.db', gs='KB9JHU')

In [3]:
ephem

IntervalTree([Interval(datetime.datetime(2017, 6, 8, 1, 12, 10, 960466), datetime.datetime(2017, 6, 8, 1, 14, 53, 187586), PassTuple(start=datetime.datetime(2017, 6, 8, 1, 12, 10, 960466), end=datetime.datetime(2017, 6, 8, 1, 14, 53, 187586), duration=162.22712, rise_az=348.20099483049904, set_az=36.51784545409412, tca=datetime.datetime(2017, 6, 8, 1, 13, 32, 60951), max_el=11.892975538504555, gs='KB9JHU', sat='ISS (ZARYA)')), Interval(datetime.datetime(2017, 6, 8, 2, 47, 39, 855197), datetime.datetime(2017, 6, 8, 2, 53, 51, 728298), PassTuple(start=datetime.datetime(2017, 6, 8, 2, 47, 39, 855197), end=datetime.datetime(2017, 6, 8, 2, 53, 51, 728298), duration=371.873101, rise_az=321.90878267718193, set_az=101.0396304297806, tca=datetime.datetime(2017, 6, 8, 2, 50, 46, 345622), max_el=36.36530683936023, gs='KB9JHU', sat='ISS (ZARYA)')), Interval(datetime.datetime(2017, 6, 8, 4, 24, 26, 622261), datetime.datetime(2017, 6, 8, 4, 29, 47, 168816), PassTuple(start=datetime.datetime(2017, 6,

In [4]:
orbital

IntervalTree([Interval(datetime.datetime(2017, 6, 8, 2, 50, 9, 229572), datetime.datetime(2017, 6, 8, 2, 51, 23, 4999), PassTuple(start=datetime.datetime(2017, 6, 8, 2, 50, 9, 229572), end=datetime.datetime(2017, 6, 8, 2, 51, 23, 4999), duration=73.775427, rise_az=4.145137839760012, set_az=58.483800835882924, tca=datetime.datetime(2017, 6, 8, 2, 50, 46, 117285), max_el=11.75032394366002, gs='KB9JHU', sat='ISS (ZARYA)'))])

In [31]:
orbital[orbital.begin()+datetime.timedelta(minutes=10)]

set()

In [50]:
import ephem
from math import degrees
import pyorbital.orbital
import datetime


satellite_name = "ISS (ZARYA)"
line1 = "1 25544U 98067A   18172.31512506  .00016717  00000-0  10270-3 0  9032"
line2 = "2 25544  51.6424 358.1382 0003248 238.2382 121.8454 15.53975871 39153"
start_time = datetime.datetime(2018, 6, 20, 14, 0, 0)


(lon, lat, ele) = (-87.0383, 41.4631, 100)

In [51]:
def sim_pyephem():
    observer = ephem.Observer()
    observer.lon = str(lon)
    observer.lat = str(lat)
    observer.elevation = ele
    #observer.date = (start_time + 2440587.5 * 86400) / 86400 - 2415020
    observer.date = start_time
    satellite = ephem.readtle(satellite_name, line1, line2)
    satellite.compute(observer)

    alt1 = float(repr(satellite.alt))
    alt1 = degrees(alt1)
    az1 = float(repr(satellite.az))
    az1 = degrees(az1)

    print("PyEphem result:  ", start_time, alt1, az1)


def sim_pyorbital():
    satellite = pyorbital.orbital.Orbital(satellite_name, line1=line1, line2=line2)
    #time1 = datetime.datetime.utcfromtimestamp(start_time)
    az1, alt1 = satellite.get_observer_look(start_time, lon, lat, ele)
    print("PyOrbital result:", start_time, alt1, az1)

sim_pyephem()
sim_pyorbital()

PyEphem result:   2018-06-20 14:00:00 -85.38464560681462 356.4038334576309
PyOrbital result: 2018-06-20 14:00:00 -85.41967282961288 356.4057832621286


In [52]:
sat = pyorbital.orbital.Orbital(satellite_name, line1=line1, line2=line2)

In [64]:
sat.get_next_passes(start_time, 24, lon, lat, ele, tol=0.1, horizon=10)[0]

(datetime.datetime(2018, 6, 20, 14, 46, 57, 768273),
 datetime.datetime(2018, 6, 20, 14, 52, 18, 610562),
 datetime.datetime(2018, 6, 20, 14, 49, 37, 682368))

In [57]:
import schedulingbazaar

In [60]:
d = schedulingbazaar.get_passes({'name':'foo', 'lat':lat, 'lon':lon, 'altitude':ele}, [satellite_name, line1, line2], start_time, duration=24)[0]
(d['start'], d['end'], d['tca'])

(datetime.datetime(2018, 6, 20, 14, 46, 19, 973362),
 datetime.datetime(2018, 6, 20, 14, 52, 56, 531628),
 datetime.datetime(2018, 6, 20, 14, 49, 37, 661854))